# A Physicist Plans a Party (Comparing Prices with Monte Carlo Simulation)

> Adam Lyon, January 2018

We're planning our daughter's Bat Mitzvah. Banquet halls offer different catering and beverage plans that we need to evaluate. It's hard to do this without knowing *a priori* who is coming and what everyone will drink. So we have to guess. A Monte Carlo simulation is one way to do the guessing. Instead of trying to guess particular situations, we'll try to guess *all* situations (or at least a large sample). We create many, many Bat Mitzvahs randomly selecting guest count and drink selections. We can then apply different the pricing rules to the parties. That will create distributions of total prices and other results that we can compare to each other. 

In this notebook, we'll examine two banquet halls WT and cpr (names changed out of paranoia). 

Speed is important here as we want to make as many guesses as we can. I'm going to try to use the [Julia Language](https://julialang.org) as it is just-in-time compiled. Exploiting [parallelization](https://docs.julialang.org/en/stable/manual/parallel-computing) is easier in Julia than in other languages. My laptop has four [hyperthreaded](https://en.wikipedia.org/wiki/Hyper-threading) cores, so I can get eight simultaneous threads going. That's a free factor of ~8 in speed! With the current code here I can generate a million Bat Mitvah's in a minute. 

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Configure-Julia" data-toc-modified-id="Configure-Julia-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Configure Julia</a></span></li><li><span><a href="#Party-Parameters" data-toc-modified-id="Party-Parameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Party Parameters</a></span><ul class="toc-item"><li><span><a href="#Define-Guests" data-toc-modified-id="Define-Guests-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Define Guests</a></span></li><li><span><a href="#Define-drinks" data-toc-modified-id="Define-drinks-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Define drinks</a></span></li><li><span><a href="#Define-costs" data-toc-modified-id="Define-costs-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Define costs</a></span></li></ul></li><li><span><a href="#Make-Parties" data-toc-modified-id="Make-Parties-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Make Parties</a></span><ul class="toc-item"><li><span><a href="#Run-single-threaded" data-toc-modified-id="Run-single-threaded-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Run single threaded</a></span></li><li><span><a href="#Restore-from-file" data-toc-modified-id="Restore-from-file-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Restore from file</a></span></li><li><span><a href="#Run-in-Parallel" data-toc-modified-id="Run-in-Parallel-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Run in Parallel</a></span></li></ul></li><li><span><a href="#Results" data-toc-modified-id="Results-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Results</a></span><ul class="toc-item"><li><span><a href="#Check-coverage" data-toc-modified-id="Check-coverage-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Check coverage</a></span></li><li><span><a href="#Distributions-of-guests" data-toc-modified-id="Distributions-of-guests-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Distributions of guests</a></span></li><li><span><a href="#Distributions-of-drinks" data-toc-modified-id="Distributions-of-drinks-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Distributions of drinks</a></span></li><li><span><a href="#Cost-results" data-toc-modified-id="Cost-results-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Cost results</a></span></li><li><span><a href="#Explore-cpr-where-package-plan-is-favored" data-toc-modified-id="Explore-cpr-where-package-plan-is-favored-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Explore cpr where package plan is favored</a></span></li></ul></li><li><span><a href="#Back-Matter" data-toc-modified-id="Back-Matter-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Back Matter</a></span></li></ul></div>

## Configure Julia

In [1]:
addprocs() ; nworkers() # Use all available threads on the machine

8

In [2]:
# Load packages ... @everywhere loads them on all the worker threads
@everywhere begin
    using Base.Test
    using Lazy: @>, @>>, @_
    using Distributions
    using Plots
    using DataFrames
    using DataFramesMeta
    using StatPlots
end

A note about the `@>` syntax you'll see below as it will likely look unfamiliar. The [Lazy.jl](https://github.com/MikeInnes/Lazy.jl) package has the `@>` and `@>>` (and other) macros to create a pipeline of commands. `@>` threads the result of a function into the first argument position of the next function on the line. `@>>` threads the result of a function into the *last* argument position of the next function (needed rarely). For example,

```julia
mean( @select(x, :nA ) )  # Take the mean of the :nA column of the data frame x
@> x @select(:nA) mean    # This line does the same as above

ndupl = sum( nonunique( @select(x, :nA, :nT, :nC, :nM, :nW, :nSA, :nSK) ) )
ndupl = @> x @select(:nA, :nT, :nC, :nM, :nW, :nSA, :nSK) nonunique sum()  # Same as line above

printfmtln("{1:.2f}%", ndupl/nrow(x)*100 )
@>> ndupl/nrow(x)*100 printfmtln("{1:.2f}%")  # Same as line above
```

I find the threading macros make complex expressions easier to read as you can follow the flow from left to right instead of from inner parentheses to outer. 

In [3]:
srand()   # Seed the global random number generator

MersenneTwister(UInt32[0x69aa356e, 0xfb5298bf, 0x0631de91, 0x563fed6d], Base.dSFMT.DSFMT_state(Int32[1846462847, 1072720844, -366537929, 1073570517, -61604218, 1072745987, -653813774, 1073048879, 1704116874, 1073065813  …  854955082, 1073737158, -1768884899, 1073614175, 1685632459, 444295841, -1975568163, -1910962729, 382, 0]), [1.99755, 1.41392, 1.47319, 1.70305, 1.99948, 1.86288, 1.65262, 1.49116, 1.55045, 1.24593  …  1.58409, 1.24684, 1.4092, 1.7795, 1.67159, 1.41496, 1.24681, 1.86014, 1.1064, 1.09842], 382)

*Assumptions and limitations:*

* We are going to ignore correlations between guests. For example, many guests come in families. We're going to ignore that for now.
* We'll assume that no one drinks beer

Party parameters from the latest [guest list](https://docs.google.com/spreadsheets/d/1XnHJmx3Ke0D1UaFNWex0c4MNv9CawyiZPqVUIqkzePY/edit#gid=0)

In [73]:
function printNicely(x) 
    """
    Julia prints unsigned integers in hex by default. Print in decimal instead
    """
    [println(aNum) for aNum in x]
    nothing
end

printNicely (generic function with 1 method)

## Party Parameters

Here is where we define the parameters of the parties

### Define Guests

In [5]:
# Ranges of guests
@everywhere begin
    const adultRange  = (41, 71)   # Adult food rate and drinks alcohol
    const teenRange   = (18, 27)   # 13 and over (adult food rate, but no alcohol)
    const childRange  = (11, 22);  # Child fee 
end

In [6]:
@everywhere function randIntInRange(range::Tuple{Int, Int})
    """
    Return a flat random distribution within the given range
    """
    
    floor(Int, rand()*(range[2]-range[1]+1)+range[1])
end

@everywhere function randomsInRanges(ranges...)
    """
    Pull a random number for each range. The output is in the same order of the ranges
    """
    [randIntInRange(r) for r in ranges]
end

### Define drinks

We need to determine how many drinks the guests are going to have. Here are the categories of drinks
* Soda
* House wine
* Mixed drink (top shelf)

Let's assume the following:

__Drinking Behavior A__

* On average, each person has a drink an hour. Some people may drink much less. Others make drink a little more. 
* Let's do the number of drinks for adults as a gaussian with mean of 4 and sd of 2. We do not allow more than 5 drinks. 
* For adults, two of their drinks have a 20% chance of being a mixed drink, 50% wine, 30% soda
* For adults, the third drink has 50% wine, 50% soda, 0% mixed drink
* For adults, the other drinks have 10% wine, 90% soda
* For kids, number of sodas is a gaussian with mean of 5 and sd of 1

Here's what a Gaussian $\mu=4; \sigma=2$ looks like (with integers; note that I've centered the bins on the integers)

In [25]:
gr()
niceHistBins(low, high) = low-0.5:high+0.5
niceHistBins(t::Tuple) = niceHistBins(t[1], t[2])
@_ round.(Int, rand(Normal(4, 2),10000)) filter(x -> x <= 5, _) histogram(_, nbin=niceHistBins(0, 8), normalize=true, legend=nothing, line=nothing)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 0.25

Here's what a Gaussian with $\mu=5; \sigma=2$ looks likes (with integers)

In [26]:
@> round.(Int, rand(Normal(5, 2),10000))  histogram(nbin=niceHistBins(1, 9), normalize=true, legend=nothing, line=nothing)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20

In [27]:
@everywhere function drinksForProb(vs::Array{Float64,1}, probMixed::Float64, probWine::Float64, probSoda::Float64)
    """
    Given a flat [0,1] random number array vs, determine if the drink is a mixed drink, wine, or soda
    according to the probabilities.
    Returns [nMixed, nWine, nSoda, 0]  # We always return soda in the third element 
    """
    @assert probMixed + probWine + probSoda ≈ 1.0
    
    nDrinks = zeros(Int, 4)
    
    nDrinks[1] += @> vs .<= probMixed sum                          # Mixed drinks
    nDrinks[2] += @> probMixed .< vs .<= probWine+probMixed sum    # Wine
    nDrinks[3] += @> probWine+probMixed .< vs sum                  # Sodas
    
    return nDrinks
end

In [28]:
# Some tests to be sure I did this right
@test drinksForProb([0.5, 0.1, 0.95], 0.7, 0.2, 0.1) == [2, 0, 1, 0]
@test drinksForProb([0.5, 0.8, 0.95], 0.7, 0.2, 0.1) == [1, 1, 1, 0]
@test drinksForProb([0.95, 0.95, 0.95], 0.7, 0.2, 0.1) == [0, 0, 3, 0]
@test drinksForProb([0.95, 0.95, 0.95], 0.8, 0.2, 0.0) == [0, 3, 0, 0]

Test Passed

In [29]:
@everywhere function drinkingBehaviorA(g::Array{Int,1})
    """
    Implement Drinking Behavior A (see above)
    g has the form [# of adults, # of teens, # of kids]
    """
    
    nDrinks = zeros(Int, 4)  # [nMixed, nWine, nSoda_adults, nSoda_kids]
    
    # Adult drinks
    nAdultDrinks = @>> round.(Int, rand(Normal(4, 2), g[1]) ) filter( x -> x <= 5)
    nDrinks += @> rand(@> nAdultDrinks .>= 1 sum) drinksForProb(0.2, 0.5, 0.3)         # Drink 1
    nDrinks += @> rand(@> nAdultDrinks .>= 2 sum) drinksForProb(0.2, 0.5, 0.3)         # Drink 2
    nDrinks += @> rand(@> nAdultDrinks .>= 3 sum) drinksForProb(0.0, 0.5, 0.5)         # Drink 3
    for i in 4:maximum(nAdultDrinks)                                                   # Drinks 4, ...
        nDrinks += @> rand(@> nAdultDrinks .>= i sum) drinksForProb(0.0, 0.10, 0.90)
    end
    
    # Kids drinks (# of sodas)
    nDrinks[4] += @> round.(Int, rand(Normal(5, 2), g[2] + g[3]) ) sum
    
    nDrinks
end

### Define costs

With the number of people and the number of drinks, we can try several different cost plans for the party.  Below are functions that create data frame entries for each "party". 

In [30]:
@everywhere begin

    function makeCommonEntry(g::Array{Int,1}, d::Array{Int,1})
        """
           Make the common entries
               w = worker thread ID,   nA = #Adults,       nT = #Teens, nC = #Children
              nM = #Mixed drinks,      nW = #Wine drinks, 
             nSA = #sodas for adults, nSK = #sodas for kinds (teens+children)
        """
        DataFrame(w=myid(), nA=g[1], nT=g[2], nC=g[3], nM=d[1], nW=d[2], nSA=d[3], nSK=d[4])
    end

    function wtEntry(g::Array{Int,1}, d::Array{Int,1})
        """
           Costing for the WT Banquet Hall
           Wfood   = Cost for food,          WdrinkT = drink cost on tab plan,     wDrinkP = drink costs on package plan
           WtotalT = total cost on tab plan, WtotalP = total cost on package plan,    WT_P = total tab - total package
        """

        wtFood(g) = 23.0(g[1]+g[2]) + 13.0g[3]                   #  Food: $23/adult, $13/child
        wtDrinkTab(g, d) = 9.0d[1] + 7.5d[2] + 3d[3] + 3d[4]     #  Drink tab: Mixed=$9, Wine=$7.50, Soda=$3
        wtDrinkPkg(g, d) = 20.0(g[1] + g[2] + g[3])              #  Drink package: $20/person

        food = wtFood(g)
        drinkT = wtDrinkTab(g, d)
        drinkP = wtDrinkPkg(g, d)

        totalT = (1.0 + 0.2 + 0.0975)*(food + drinkT)                      # Tax is 9.75% and service is 20%
        totalP = (1.0 + 0.2 + 0.0975)*(food + drinkP)

        DataFrame(Wfood=food, WdrinkT=drinkT, WdrinkP=drinkP, WtotalT=totalT, WtotalP=totalP, WT_P=totalT-totalP)
    end

    function cprEntry(g::Array{Int,1}, d::Array{Int,1})
        """
           Costing for the cpr Banquet Hall
           Cfood   = Cost for food,          CdrinkT = drink cost on tab plan,     CDrinkP = drink costs on package plan
           CtotalT = total cost on tab plan, CtotalP = total cost on package plan,    CT_P = total tab - total package
        """

        cprFood(g) = 27.0*(g[1] + g[2]) + 15.0*g[3]                        # Food is $27/adult, $15/child
        cprDrinkTab(g, d) = 9.0*d[1] + 7.0*d[2] + 1.0*d[3] + 1.0*d[4]      # Drink tab: Mixed=$9, Wine=$7, Soda=$1
        cprDrinkPkg(g, d) = 3.0*(g[1]+g[2]+g[3]) + 9.0*d[1] + 7.0*d[2]     # Unlimited soda service $3/person 

        food = cprFood(g)
        drinkT = cprDrinkTab(g, d)
        drinkP = cprDrinkPkg(g, d)

        totalT = (1.0 + 0.2 + 0.1075)*(food + drinkT) + 500.0
        totalP = (1.0 + 0.2 + 0.1075)*(food + drinkP) + 500.0

        DataFrame(Cfood=food, CdrinkT=drinkT, CdrinkP=drinkP, CtotalT=totalT, CtotalP=totalP, CT_P=totalT-totalP)
    end
    
end

I have a spreadsheet with some pre-worked out examples. Let's check them. 

In [31]:
wtEntry([50, 25, 15], [50, 150, 100, 30])

,Wfood,WdrinkT,WdrinkP,WtotalT,WtotalP,WT_P
1,1920.0,1965.0,1800.0,5040.79,4826.7,214.087


In [32]:
cprEntry([50, 25, 15], [50, 150, 100, 30])

,Cfood,CdrinkT,CdrinkP,CtotalT,CtotalP,CT_P
1,2250.0,1630.0,1770.0,5573.1,5756.15,-183.05


## Make Parties

Time to party!

In [33]:
@everywhere function makeParty()
    """Make one party"""
    
    # Determine guests
    g = randomsInRanges(adultRange, teenRange, childRange)
    
    # Determine drinks
    d = drinkingBehaviorA(g)
    
    # Common entry
    c = makeCommonEntry(g, d)
    
    # Cost at WT
    c = hcat(c, wtEntry(g, d))
        
    # Cost at CPR
    c = hcat(c, cprEntry(g, d))
    
    c
    
end 

In [35]:
@time makeParty()

  0.000237 seconds (482 allocations: 120.328 KiB)


,w,nA,nT,nC,nM,nW,nSA,nSK,Wfood,WdrinkT,WdrinkP,WtotalT,WtotalP,WT_P,Cfood,CdrinkT,CdrinkP,CtotalT,CtotalP,CT_P
1,1,58,27,12,15,63,59,192,2111.0,1360.5,1940.0,4504.27,5256.17,-751.901,2475.0,827.0,867.0,4817.36,4869.66,-52.3


**Choose from three options for how to proceed**

### Run single threaded

In [36]:
x = makeParty()
@time for i in 1:125000-1
    append!(x, makeParty())
end

 31.212703 seconds (65.01 M allocations: 14.572 GiB, 11.30% gc time)


### Restore from file

In [40]:
using JLD
function dataFromCache(filename)
    if isfile(filename)
        return JLD.load(filename)["x"]
    end
    return nothing
end

dataFromCache (generic function with 1 method)

In [41]:
x = dataFromCache("party1M.jld");

### Run in Parallel

In [40]:
@time x = @parallel (append!) for i = 1:nworkers()
    y = makeParty()
    for i in 1:125000-1
        append!(y, makeParty())
    end
    y
end ;

 57.742994 seconds (257.90 k allocations: 314.504 MiB, 0.91% gc time)


In [41]:
using JLD
@save "party1M.jld" x

## Results

The result is a [DataFrame](http://juliadata.github.io/DataFrames.jl/stable/) with information about each party. Let's just look at the top few rows.

In [42]:
head(x)

,w,nA,nT,nC,nM,nW,nSA,nSK,Wfood,WdrinkT,WdrinkP,WtotalT,WtotalP,WT_P,Cfood,CdrinkT,CdrinkP,CtotalT,CtotalP,CT_P
1,2,66,23,15,19,84,93,191,2242.0,1653.0,2080.0,5053.76,5607.79,-554.032,2628.0,1043.0,1071.0,5299.83,5336.44,-36.61
2,2,71,18,20,19,73,76,212,2307.0,1582.5,2180.0,5046.63,5821.88,-775.256,2703.0,970.0,1009.0,5302.45,5353.44,-50.9925
3,2,71,20,16,20,67,72,168,2301.0,1402.5,2140.0,4805.29,5762.2,-956.906,2697.0,889.0,970.0,5188.69,5294.6,-105.908
4,2,58,23,22,14,56,73,256,2149.0,1533.0,2060.0,4777.39,5461.18,-683.783,2517.0,847.0,827.0,4898.43,4872.28,26.15
5,2,59,27,16,16,63,59,184,2186.0,1345.5,2040.0,4582.12,5483.23,-901.114,2562.0,828.0,891.0,4932.42,5014.8,-82.3725
6,2,47,21,15,12,57,65,179,1759.0,1267.5,1660.0,3926.88,4436.15,-509.269,2061.0,751.0,756.0,4176.69,4183.23,-6.5375


How many parties did we get?

In [43]:
using Formatting

@> nrow(x) format(commas=true) println

1,000,000


How many rows are duplications? If there are a lot, that could mean we've oversampled. 

In [44]:
# Look for duplicate rows. We need to remove the w column as that's the worker node ID
ndupl = @> x @select(:nA, :nT, :nC, :nM, :nW, :nSA, :nSK) nonunique sum()

263

In [45]:
@>> ndupl/nrow(x)*100 printfmtln("{1:.2f}%")

0.03%


I think this is ok.

### Check coverage

We need to look at coverage (did we cover a large sample of possibilities?). How many possible set of guests should we have (assuming no correlations)?

In [46]:
tdiff(t::Tuple{Int,Int}) = t[2]-t[1]+1
totalGuestSetsExpected = tdiff(adultRange) * tdiff(teenRange) * tdiff(childRange)

3720

How many sets of guests did we generate?

In [47]:
gs = @> x @select(:nA, :nT, :nC)          # Select out the columns involving guests
@> nrow(gs) format(commas=true) println

1,000,000


In [48]:
totalGuestSets = @> gs unique nrow    # Get the unique set (remove all duplicates)

3720

In [49]:
# They should agree with what we expect
@test totalGuestSetsExpected == totalGuestSets

Test Passed

We now want to see how many samples there are for each guest set. To do this, we'll make a unique hash for each row of guest sets and see how often each appears in the full set of parties. 

In [50]:
gsHashes = [ hash(r) for r in eachrow(gs) ]
nHashes = @> gsHashes unique length

3720

In [51]:
@test nHashes == totalGuestSets # This should match the total number of guest sets

Test Passed

In [52]:
using FreqTables

In [53]:
gsFreq = freqtable(gsHashes) # Determine how often a hash (guest set) appears 
gsFreq[1:5]  # Just show the first five

5-element Named Array{Int64,1}
Dim1               │ 
───────────────────┼────
0x00023f9b13bb2633 │ 265
0x0012f8d42691d831 │ 263
0x00145e2cafb86bff │ 278
0x00321dc9a184eafe │ 265
0x003d384ad9002082 │ 319

In [54]:
histogram(gsFreq.array, title="Number of drink sets for guest set", xlab="Number of drink sets", legend=nothing, line=false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 225 
 
 
 250 
 
 
 275 
 
 
 300 
 
 
 325 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 Number of drink sets for guest set 
 
 
 Number of drink sets

In [55]:
length(gsFreq)

3720

In [56]:
@> sum(gsFreq) format(commas=true) println  # Should be the number entries in the whole list

1,000,000


In [57]:
mean(gsFreq), minimum(gsFreq), maximum(gsFreq)

(268.81720430107526, 216, 326)

The [histogram](https://en.wikipedia.org/wiki/Histogram) above is a distribution of the number of drink sets per guest set (this is the distribution of the number of times a guest set appears in the list, presumably with different drinks). In order to cover a large number of drink situations, we want this distribution to go out far (going out too far leads to duplication and unnecessary parties). With 100,000 parties, the plot has a mean of 27. With 1M parties we get a mean of 268. The Gaussian distribution in the plot is expected from the [Central Limit Theorem](https://en.wikipedia.org/wiki/Central_limit_theorem) [it works!]. Note that this plot used to look stratified with three distinct peaks, and the distribution of guests had the edge bins at half the height as the rest. That was because I originally had the random number generation for the number of guests calling `round` instead of `floor`. Using `round` means that the values at the ends of the range get hit only half as often -- a good catch by this test. 

### Distributions of guests 
Who did we invite? Let's look at [histograms](https://en.wikipedia.org/wiki/Histogram) that display distributions of interesting results from the parties. For the number of guests, we generated flat distributions and that's what we should see.

In [58]:
h1 = @df x histogram( :nA, legend=false, nbins=niceHistBins(adultRange), title="#Adults")
h2 = @df x histogram( :nT, legend=false, nbins=niceHistBins(teenRange), title="#Teens")
h3 = @df x histogram( :nC, legend=false, nbins=niceHistBins(childRange), title="#Children")
plot(h1, h2, h3, layout=(3,1))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 70 
 
 
 0 
 
 
 10000 
 
 
 20000 
 
 
 30000 
 
 
 #Adults 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 18 
 
 
 20 
 
 
 22 
 
 
 24 
 
 
 26 
 
 
 0 
 
 
 25000 
 
 
 50000 
 
 
 75000 
 
 
 100000 
 
 
 #Teens 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 12.5 
 
 
 15.0 
 
 
 17.5 
 
 
 20.0 
 
 
 22.5 
 
 
 0 
 
 
 20000 
 
 
 40000 
 
 
 60000 
 
 
 80000 
 
 
 #Children

Are there any correlations?

In [59]:
p1 = @df x histogram2d( :nA, :nT, nbin=15, xlab="Number of adults", ylab="Number of teens")
p2 = @df x histogram2d( :nA, :nC, nbin=15, xlab="Number of adults", ylab="Number of children")
p3 = @df x histogram2d( :nT, :nC, nbin=15, xlab="Number of teens", ylab="Number of children")
plot(p1, p2, p3, layout=3)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 70 
 
 
 18 
 
 
 20 
 
 
 22 
 
 
 24 
 
 
 26 
 
 
 28 
 
 
 Number of adults 
 
 
 Number of teens 
 
 
<image width="166" height="148" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAKYAAACUCAYAAAAZOlLRAAAEtElEQVR4nO3dzW5bRRjG8XfOObbj
1M5H0wSCotIWNSpFAhZsQN2wQ6xhAZfAfXA9rNiCEGLZsqCiYhGVlrYpmFhRE3/bM9zCYyGkB+n/
W7+asZp/z2o0k6qqX0J0fvaVNJfGL9QlI+3cleZKWUlz9fOf5L1XR/e0wYsn0ljpXpP3julQGkur
ubxk6R1pa46eyWuqmt33pLnF9FSaq/7NjwH+K4QJS4QJS4QJS4QJS4QJS4QJS4QJS4QJS4QJS4QJ
S4QJS4QJS4QJS03OE326qrW5rB1Ri4jIk5faYHtLGlvuHct7N09/0NY8+khbMK3x/7xqtCX7N/Ul
X2hH/lZb4vG41VTeezF5rg2Kx/j4YsISYcISYcISYcISYcISYcISYcISYcISYcISYcISYcISYcIS
YcISYcJSSmlDvu3t4uRTaS7vvS3/gPbJd9Lc4vU70lzp7Mh7y8fz6o40lmbaDW4REaW9xu8UqfuX
zde09SZ/65vPL7W9e4fSHF9MWCJMWCJMWCJMWCJMWCJMWCJMWCJMWCJMWCJMWCJMWCJMWCJMWCJM
WGqqqisP5+0b0lyqN/Q1734uzZXpQNt7eibvXTauaoPzC20uL+S9o7MtjaXhI33Nuq2t+Up8tHWN
I4RVXmprctsb/s8IE5YIE5YIE5YIE5YIE5YIE5YIE5YIE5YIE5YIE5YIE5YIE5YIE5aanEfycLrU
HrIsrZ685nL0h7b3+E9twdamvHdq9aW5pvuGNLcYacfJIiKq1JLmivgYakRE2bohzdUv70tzWX3U
NiLK/vvSXDr/TZrjiwlLhAlLhAlLhAlLhAlLhAlLhAlLhAlLhAlLhAlLhAlLhAlLhAlLTRT5KUnZ
Zv9Ynh0PH0hzpf+mtmDST8TUlXb51+LiRNt6OZb3rga/SnPLww/kNWM1k8by5p40t9a7nJG1MfHv
yBcTlggTlggTlggTlggTlggTlggTlggTlggTlggTlggTlggTlggTlggTlpqq0i+hKlcOpbnx8Gf9
F7TFNxUv1fcPd+WtSxEv/xLfP0yzV/LerdtfSHP592/kNaPSLurKPe3vuM67nE3/LWluMXsszfHF
hCXChCXChCXChCXChCXChCXChCXChCXChCXChCXChCXChCXChCXChKVUV335urfzZ59pg3Vb/gFF
vZ2tc1XbevCLvHfuaMfe0ly7xS1vizfSRURk8SjdRD96VnZuS3Pqv9Fq95a8dxqdaoPi25h8MWGJ
MGGJMGGJMGGJMGGJMGGJMGGJMGGJMGGJMGGJMGGJMGGJMGGJMGGpKaE/ctre/1Cam59+r/+C/nVp
rOlck+ZWbf32urQQj7MdvKutl7QjXRERTUu75W7RPJXXjKR9Z3LvQJprrmh/m4iI1LspzZXH30pz
fDFhiTBhiTBhiTBhiTBhiTBhiTBhiTBhiTBhiTBhiTBhiTBhiTBhiTBhqUmR5OHZ8IE0l5quvGY9
eCjNLQ+035nEo18REenwnjTX7exLc+XHr+W9p+98Is01g0fymq3jL6W5yeQvcUX9SOTyTHvYNl3/
WJrjiwlLhAlLhAlLhAlLhAlLhAlLhAlLhAlLhAlLhAlLhAlLhAlLhAlL/wA0htC35XHMAQAAAABJ
RU5ErkJggg==
" transform="translate(55, 12)"/>
 
 
 
 
 
 
 
<image width="18" height="148" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAABIAAACUCAYAAACeJ3mSAAABW0lEQVRogd3a23EDQQhE0V57nH8S
ztEvBpzE+aCkAG7RFxitdvRUf07A5/SU4OT0/UagRqBZF20UKAzUyFHqC4FUtOf+GFD6GtBTvwh0
EWhhtPpTIOaoDWifozCQc8TOo23RUmjXcpkjVdE+R0ZRDnqGyIlRlBOjKGfqQaD7ZkDpdRXVuwHN
qGiFos1V0RY6UhWpOWom260IqqiVo2kme1vXnCO1Ii6a6tplK+IcqTlSjli0y3ZtH4i1n012o8l2
XdvnaJhs5ohwYEXMEWv/XXceQUeEk9Nsstd1beEcsZ8QBJOcUg/s+xzB9hMOjFZsjlw0BHLRCCY5
pV6NwRUhHPkQ8bog6MiQFjqCxwhyVEGgYRWtc3TRS//TSnYNqohFKwUaVtGDLqEK3UItjHZVtAq6
px10L+a6dlW0ZiC1/e5fYy8cbVg0doygFdnnKIOijZqjQcfIk3yQL/9/fmk1aCz3ydQAAAAASUVO
RK5CYII=
" transform="translate(232, 12)"/>
 
 
 3500 
 
 
 4000 
 
 
 4500 
 
 
 5000 
 
 
 5500 
 
 
 6000 
 
 
 6500 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 70 
 
 
 12.5 
 
 
 15.0 
 
 
 17.5 
 
 
 20.0 
 
 
 22.5 
 
 
 Number of adults 
 
 
 Number of children 
 
 
<image width="166" height="148" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAKYAAACUCAYAAAAZOlLRAAAFQklEQVR4nO3dzW4bVRjG8fecGX/l
q05CnNJPiJAQFZWQWCE2iGtgy61xE+zYI1GJrwUbClUqpU2CkjSJsU2czMzhFp5somfx/61fnddu
//FmxuM06D0sITp7+YU0V3oj9cgo/Q1prpoeaOflWt4dRXvrZTSW5qrpoby6mTzXzhzuymd2Jz+L
k0nbPT+Rd7dPv5bm8tELbU7eDNwhwoQlwoQlwoQlwoQlwoQlwoQlwoQlwoQlwoQlwoQlwoSllPOa
fBPHxfG32qFdK7+A6uyVNNc8/lI77/gXeXfpDbW5qq8dWA/k3elmoe2+xZkh/rvn5Vw7bqjdYBMR
UV+8keaazUfSHJ+YsESYsESYsESYsESYsESYsESYsESYsESYsESYsESYsHSLpwNE1Kd/S3PN7qfy
mc3kE22wvdbmkv63ltobbVB8iEJ9/Ke8u9kSrxkvzuUzu5Vtaa4df6gd2C713cN1aa4MtNfIJyYs
ESYsESYsESYsESYsESYsESYsESYsESYsESYsESYspZT68vfKb7rvpLnF+W/6K+gaaSwtL6S5+mxf
Xl362o8YNDvPpLnqnXYvQUREuta+V96uT+Qz1Yf+5+uZNNet6rvL6D1t8EbbzScmLBEmLBEmLBEm
LBEmLBEmLBEmLBEmLBEmLBEmLBEmLBEmLKWUhvJNHFc/aTczLPe0B/lHRPTe/irNdatb2oHiTSER
EWWwJs2pDx1on3wl747Lv7Td81P5yHb7Y3Gy08aaK3l3kh+OIN5oIm8G7hBhwhJhwhJhwhJhwhJh
whJhwhJhwhJhwhJhwhJhwlKdxGuXERHN+L40l+fH8pllsCrNpUa7Fttu7Mq789Zn2pmdds24zA7k
3Sn3pLm80B70EBGRnjyW5tqz37UD66G+ezm

There are no correlations among the guests. That's good, as we didn't put any in.

### Distributions of drinks

What did everyone drink?

In [60]:
@df x histogram( [:nM, :nW, :nSA, :nSK], layout=4, legend=false, nbins=15, title=["Mixed Drinks" "Wine" "Soda Adults" "Soda Kids"], 
    title_location=:center, line=nothing,
    color=[:green :darkgreen :darkblue :lightblue])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0 
 
 
 100000 
 
 
 200000 
 
 
 300000 
 
 
 Mixed Drinks 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 0 
 
 
 50000 
 
 
 100000 
 
 
 150000 
 
 
 200000 
 
 
 250000 
 
 
 Wine 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 125 
 
 
 0 
 
 
 50000 
 
 
 100000 
 
 
 150000 
 
 
 200000 
 
 
 250000 
 
 
 Soda Adults 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 150 
 
 
 200 
 
 
 250 
 
 
 300 
 
 
 0 
 
 
 50000 
 
 
 100000 
 
 
 150000 
 
 
 200000 
 
 
 250000 
 
 
 Soda Kids

Easier to think about number of drinks per person (a kid is a teen or child)

In [61]:
@df x histogram( [:nM./:nA :nW./:nA :nSA./:nA :nSK./(:nT.+:nC) ], line=nothing,
                color=[:green :darkgreen :darkblue :lightblue],
                layout=4, nbins=20, label="", title=["mixed drinks/adult" "wine/adult" "soda/adult" "soda/kid"] )

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0 
 
 
 50000 
 
 
 100000 
 
 
 150000 
 
 
 200000 
 
 
 250000 
 
 
 mixed drinks/adult 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
 1.25 
 
 
 1.50 
 
 
 1.75 
 
 
 0 
 
 
 100000 
 
 
 200000 
 
 
 300000 
 
 
 wine/adult 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 0 
 
 
 50000 
 
 
 100000 
 
 
 150000 
 
 
 200000 
 
 
 soda/adult 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.5 
 
 
 4.0 
 
 
 4.5 
 
 
 5.0 
 
 
 5.5 
 
 
 6.0 
 
 
 6.5 
 
 
 0 
 
 
 50000 
 
 
 100000 
 
 
 150000 
 
 
 200000 
 
 
 soda/kid

### Cost results

How do the total costs compare? Let's try a [Box plot](https://en.wikipedia.org/wiki/Box_plot)

In [62]:
@df x boxplot( ["cpr Package" "cpr Tab" "WT Package" "WT Tab"], [ :CtotalP :CtotalT :WtotalP :WtotalT], 
               ylab="Cost \$", ylim=(3000, 8000), legend=false )

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cpr Package 
 
 
 cpr Tab 
 
 
 WT Package 
 
 
 WT Tab 
 
 
 3000 
 
 
 4000 
 
 
 5000 
 
 
 6000 
 
 
 7000 
 
 
 8000 
 
 
 Cost $

Let's look at the distributions too...

In [63]:
histogram( [x[:CtotalP], x[:CtotalT], x[:WtotalP], x[:WtotalT]], bins=40, 
            layout=grid(4,1), color=[:red :blue :green :yellow], xlim=(3000, 6500), line=nothing,
            labels=["cpr Package" "cpr Tab" "WT Package" "WT Tab"])  # Note no commas in colors

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3000 
 
 
 3500 
 
 
 4000 
 
 
 4500 
 
 
 5000 
 
 
 5500 
 
 
 6000 
 
 
 6500 
 
 
 0 
 
 
 10000 
 
 
 20000 
 
 
 30000 
 
 
 40000 
 
 
 50000 
 
 
 60000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cpr Package 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3000 
 
 
 3500 
 
 
 4000 
 
 
 4500 
 
 
 5000 
 
 
 5500 
 
 
 6000 
 
 
 6500 
 
 
 0 
 
 
 10000 
 
 
 20000 
 
 
 30000 
 
 
 40000 
 
 
 50000 
 
 
 60000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cpr Tab 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3000 
 
 
 3500 
 
 
 4000 
 
 
 4500 
 
 
 5000 
 
 
 5500 
 
 
 6000 
 
 
 6500 
 
 
 0 
 
 
 10000 
 
 
 20000 
 
 
 30000 
 
 
 40000 
 
 
 50000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 WT Package 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3000 
 
 
 3500 
 
 
 4000 
 
 
 4500 
 
 
 5000 
 
 
 5500 
 
 
 6000 
 
 
 6500 
 
 
 0 
 
 
 10000 
 
 
 20000 
 
 
 30000 
 
 
 40000 
 
 
 50000 
 
 
 60000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 WT Tab

Let's compare the drink packages to paying by drink (tab)...

In [64]:
h1 = @df x histogram( :CT_P, nbin=40, legend=false, title="cpr: How much more Tab is compared to Package", line=nothing)
h1 = vline!([0])
h2 = @df x histogram( :WT_P, nbin=40, legend=false, title="WT: How much more Tab is compared to Package", 
                                      xlab="Price difference \$", line=nothing)
plot(h1, h2, layout=(2,1))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -150 
 
 
 -100 
 
 
 -50 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 0 
 
 
 25000 
 
 
 50000 
 
 
 75000 
 
 
 100000 
 
 
 cpr: How much more Tab is compared to Package 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1250 
 
 
 -1000 
 
 
 -750 
 
 
 -500 
 
 
 -250 
 
 
 0 
 
 
 25000 
 
 
 50000 
 
 
 75000 
 
 
 100000 
 
 
 125000 
 
 
 WT: How much more Tab is compared to Package 
 
 
 Price difference $

Positive means that the price of putting beverages on a tab is *more* than the cost of the package. So using a tab is bad in those cases. Negative means that tab is *less* expensive than the package. 

In the above, we see that for WT, the tab plan is always cheaper than the package beverage plan. Beverage consumption does not make t

For cpr, the tab plan is usually better. Let's look at where the package plan is advantageous (right of the red line on the plot). 

### Explore cpr where package plan is favored

For cpr, the tab price is usually cheaper than the package price, and so the by drink (tab pricing) is the better option.

There are some parties where the opposite is true ... the package price beats the tab price (the positive part of the plot). What fraction of parties have this situation?

In [65]:
ctc = @where x (:CT_P .> 0);   # ctm = "cpr tab compare"
percPkgBest = @>> nrow(ctc) / nrow(x) *100  printfmtln("{1:.1f}%")

16.6%


So for a little more than 16% of the simulated parties, the tab cost is less expensive than the package cost. What's special about those parties?

In [66]:
@df ctc histogram( [ :nA, :nT, :nC ], layout=(1,3), legend=false, nbins=15, title=["#Adults" "#Teens" "#Children"], 
    title_location=:left, line=nothing,
    color=[:red :darkred :pink])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 70 
 
 
 0 
 
 
 10000 
 
 
 20000 
 
 
 30000 
 
 
 #Adults 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20.0 
 
 
 22.5 
 
 
 25.0 
 
 
 27.5 
 
 
 0 
 
 
 5000 
 
 
 10000 
 
 
 15000 
 
 
 20000 
 
 
 25000 
 
 
 #Teens 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 12.5 
 
 
 15.0 
 
 
 17.5 
 
 
 20.0 
 
 
 22.5 
 
 
 0 
 
 
 5000 
 
 
 10000 
 
 
 15000 
 
 
 20000 
 
 
 25000 
 
 
 #Children

Hmm. These seem to be parties where there are few adults and lots of kids.

In [67]:
p1 = @df ctc histogram2d( :nA, :nT, nbin=15, xlab="Number of adults", ylab="Number of teens")
p2 = @df ctc histogram2d( :nA, :nC, nbin=15, xlab="Number of adults", ylab="Number of children")
p3 = @df ctc histogram2d( :nT, :nC, nbin=15, xlab="Number of teens",  ylab="Number of children")
plot(p1, p2, p3, layout=3)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 70 
 
 
 18 
 
 
 20 
 
 
 22 
 
 
 24 
 
 
 26 
 
 
 28 
 
 
 Number of adults 
 
 
 Number of teens 
 
 
<image width="166" height="148" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAKYAAACUCAYAAAAZOlLRAAAEiElEQVR4nO3dzY4UVRjG8aeHEiMo
GomYuGCtLrwJF16FF+a9cAPEwMIFymIChM4M0zP9Uf1d1VV1WHADTyVqnk7+v/Wb08zMn169OWfy
8tffiky/PPvdmjv99Yd7pKrnf1pz9Yun1tzrVz/an/1m+dia+7v+0pp7u53Yn/222Vtzd5OFfeZK
H6y5fe+deRq8f6Mkdb03W8rJmruwPxn4HxEmIhEmIhEmIhEmIhEmIhEmIhEmIhEmIhEmIhEmIhEm
IhEmIhEmIlWbwwN7eH/33Jp7cPnKPrN5/5U1V8+8FbXViJ/n/c6bvTl662zLU2d/9l5Ha243Wdln
tv3OmuuGgzXXm3OSVEprzrH2hjNGmIhEmIhEmIhEmIhEmIhEmIhEmIhEmIhEmIhEmIhEmIhEmIhE
mIhUbdvP7eH70xfWXP/OvyVsefmTNffu+gdr7mrn3cwmSbPjPWuubr0L8daDt/olSesLb52tKVv7
zHbw1t56c0VtMFfUJKloMOe83yXfmIhEmIhEmIhEmIhEmIhEmIhEmIhEmIhEmIhEmIhEmIhEmIhE
mIhEmIhUtX3lD1/+Y801c+8GN0narr01tVXzhTV3ffDX+Lbm5Wzrrrfmbkc8SHqUt87mrrJJUj80
1tzgrucVb5Xt06z3OxJrbzhnhIlIhIlIhIlIhIlIhIlIhIlIhIlIhIlIhIlIhIlIhIlIhIlIhIlI
1bLx18SGK+8Wt8Xlz/aZ07sn1ty1+SDpovFucJOkeeOtda3Mxz4390asvQ1ra859uFSS+uI9nFqK
t+/nzkn+bW+sveGsESYiESYiESYiESYiESYiESYiESYiESYiESYiESYiESYiESYiVev2M3v48OZ7
a25TP7LPvN09tOZujvetuZm3YCNJWnTe5VL1RW3NdcW71EqSmt48c/Df5XTffixyt4a8TaBRs4Xt
IpwxwkQkwkQkwkQkwkQkwkQkwkQkwkQkwkQkwkQkwkQkwkQkwkQkwkSkat/7l1Ctrr6z5m6W39pn
3hy9NyIXrfd/yH33UZIWk401tzHfiDx0S/uzB/tiK//nsS/BMlfP/Iuy/DO5VAtnjTARiTARiTAR
iTARiTARiTARiTARiTARiTARiTARiTARiTARiTARqVq0I95evHtszU3XX9tnXu292+ZmB29datX7
N65tL7z3HA+DdzNbP/iffTLfiCzmDW6fhr01tSJ3lW7MbW//7ol8YyISYSISYSISYSISYSISYSIS
YSISYSISYSISYSISYSISYSISYSISYSJSVXvvfEqSPtTfWHPTvXeDmyTNm4k1tzx561/zycr+7G2Z
W3Pu46Vd8V9YdR8aHXPjmj1rrseNUXTy/pCqrM03vjERiTARiTARiTARiTARiTARiTARiTARiTAR
iTARiTARiTARiTARiTARqdqb72JK0nT7yJqbHSv7zNvGu3lsJu/GtfXEW2WTpLY/WHPd4K2z9eac
JH/1bMQjp//F7Ww+b53NxTcmIhEmIhEmIhEmIhEmIhEmIhEmIhEmIhEmIhEmIhEmIhEmIhEmIn0E
DBJYck7VPGQAAAAASUVORK5CYII=
" transform="translate(55, 12)"/>
 
 
 
 
 
 
 
<image width="18" height="148" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAABIAAACUCAYAAACeJ3mSAAABW0lEQVRogd3a23EDQQhE0V57nH8S
ztEvBpzE+aCkAG7RFxitdvRUf07A5/SU4OT0/UagRqBZF20UKAzUyFHqC4FUtOf+GFD6GtBTvwh0
EWhhtPpTIOaoDWifozCQc8TOo23RUmjXcpkjVdE+R0ZRDnqGyIlRlBOjKGfqQaD7ZkDpdRXVuwHN
qGiFos1V0RY6UhWpOWom260IqqiVo2kme1vXnCO1Ii6a6tplK+IcqTlSjli0y3ZtH4i1n012o8l2
XdvnaJhs5ohwYEXMEWv/XXceQUeEk9Nsstd1beEcsZ8QBJOcUg/s+xzB9hMOjFZsjlw0BHLRCCY5
pV6NwRUhHPkQ8bog6MiQFjqCxwhyVEGgYRWtc3TRS//TSnYNqohFKwUaVtGDLqEK3UItjHZVtAq6
px10L+a6dlW0ZiC1/e5fYy8cbVg0doygFdnnKIOijZqjQcfIk3yQL/9/fmk1aCz3ydQAAAAASUVO
RK5CYII=
" transform="translate(232, 12)"/>
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 2000 
 
 
 2500 
 
 
 3000 
 
 
 3500 
 
 
 4000 
 
 
 4500 
 
<polyline clip-path="url(#clip1300)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 250.315,160.065 250.315,143.691 256.315,143.691 250.315,143.691 250.315,127.252 256.315,127.252 250.315,127.252 250.315,110.812 256.315,110.812 250.315,110.812 
 250.315,94.3719 256.315,94.3719 250.315,94.3719 250.315,77.932 256.315,77.932 250.315,77.932 250.315,61.4922 256.315,61.4922 250.315,61.4922 250.315,45.0524 
 256.315,45.0524 250.315,45.0524 250.315,28.6125 256.315,28.6125 250.315,28.6125 250.315,12.1727 256.315,12.1727 250.315,12.1727 250.315,11.811 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 70 
 
 
 12.5 
 
 
 15.0 
 
 
 17.5 
 
 
 20.0 
 
 
 22.5 
 
 
 Number of adults 
 
 
 Number of children 
 
 
<image width="166" height="148" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAKYAAACUCAYAAAAZOlLRAAAE/UlEQVR4nO3dTW4jdRdG8evEdFB/
8CVBC4RgzjoQ62CBbKDFgAFTJo2EENBkQDrpOHHsuO24XGVXFQM2cCy10DM4v/FVVb/hvDX5V11P
fv3u27Ggb158j+ba8x/oJevs5c9obv3Th2juzd9f4Xu/vPgazb3ePkFzv6+m+N5XzQHNXdYdvubt
5ArN7foVmuuGDb53d1ijuXHcobkTfGfpf2SYimSYimSYimSYimSYimSYimSYimSYimSYimSYimSY
ijRdbp7h4e36TzR3NjvH19z/NqC5zfxjNPd6/im+97I7Q3PXDXs5Y972/N5Dg+Y2p2/xNQ8De0Gi
69kLF/vhAd+bvpwxFntnyCemIhmmIhm

Indeed, these are cases where you have the minimum of adults and the maximum of kids. Let's look some more. Are we picking the parties that are skewed
to low drink consumption among adults and high consumption among kids? 

In [68]:
using StatsBase
function siHist(v1, v2, nbins1, nbins2, title)
    """
    Superimpose two histograms, one filled and one open
    """
    
    p = histogram( v1, nbins=nbins1, normalized=true, label="", line=nothing, title=title)
    
    f = fit(Histogram, v2, nbins=nbins2, closed=:left) ; f = normalize(f)
    p = plot!(f, seriestype=:step, normalized=true, label="")
end
    

siHist (generic function with 1 method)

Let's compare what these people drink (filled histogram) to the whole set (open histogram)

In [69]:
p1 = siHist( ctc[:nM], x[:nM], 15, 15, "Mixed Drinks")
p2 = siHist( ctc[:nW], x[:nW], 15, 15, "Wine")
p3 = siHist( ctc[:nSA], x[:nSA], 15, 18, "Soda Adults")
p4 = siHist( ctc[:nSK], x[:nSK], 15, 15, "Soda Kids")
plot(p1, p2, p3, p4, layout=4)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 Mixed Drinks 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 0.00 
 
 
 0.01 
 
 
 0.02 
 
 
 0.03 
 
 
 0.04 
 
 
 Wine 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 125 
 
 
 0.00 
 
 
 0.01 
 
 
 0.02 
 
 
 0.03 
 
 
 Soda Adults 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip1504)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 38.8117,375.813 38.8117,375.813 56.6417,375.813 56.6417,375.516 74.4717,375.516 74.4717,368.171 92.3016,368.171 92.3016,330.468 110.132,330.468 110.132,280.32 
 127.962,280.32 127.962,262.932 145.792,262.932 145.792,278.997 163.622,278.997 163.622,318.301 181.452,318.301 181.452,354.962 199.281,354.962 199.281,371.639 
 217.111,371.639 217.111,375.363 234.941,375.363 234.941,375.787 252.771,375.787 252.771,375.812 270.601,375.812 270.601,375.813 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 150 
 
 
 200 
 
 
 250 
 
 
 300 
 
 
 0.000 
 
 
 0.005 
 
 
 0.010 
 
 
 0.015 
 
 
 Soda Kids

Again, it's easier to look at drinks per person

In [70]:
p1 = siHist( ctc[:nM]./ctc[:nA], x[:nM]./x[:nA], 15, 15, "Mixed Drinks/Adult")
p2 = siHist( ctc[:nW]./ctc[:nA], x[:nW]./x[:nA], 15, 15, "Wine/Adult")
p3 = siHist( ctc[:nSA]./ctc[:nA], x[:nSA]./x[:nA], 15, 18, "Soda/Adult")
p4 = siHist( ctc[:nSK]./(ctc[:nT].+ctc[:nC]), x[:nSK]./(x[:nT].+x[:nC]), 15, 15, "Soda/Kid")
plot(p1, p2, p3, p4, layout=4)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 Mixed Drinks/Adult 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip1702)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.1504,175.813 39.1504,175.776 55.6462,175.776 55.6462,174.813 72.142,174.813 72.142,165.949 88.6378,165.949 88.6378,132.298 105.134,132.298 105.134,70.5303 
 121.629,70.5303 121.629,31.4961 138.125,31.4961 138.125,66.6649 154.621,66.6649 154.621,124.515 171.117,124.515 171.117,157.742 187.613,157.742 187.613,171.759 
 204.109,171.759 204.109,174.902 220.604,174.902 220.604,175.706 237.1,175.706 237.1,175.795 253.596,175.795 253.596,175.811 270.092,175.811 270.092,175.813 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
 1.25 
 
 
 1.50 
 
 
 1.75 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 Wine/Adult 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip1703)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 326.437,175.813 326.437,175.8 344.043,175.8 344.043,175.536 361.649,175.536 361.649,172.781 379.255,172.781 379.255,158.632 396.861,158.632 396.861,116.144 
 414.467,116.144 414.467,70.439 432.073,70.439 432.073,31.4961 449.679,31.4961 449.679,86.8254 467.285,86.8254 467.285,137.648 484.891,137.648 484.891,165.752 
 502.497,165.752 502.497,173.926 520.103,173.926 520.103,175.525 537.709,175.525 537.709,175.79 555.315,175.79 555.315,175.813 572.92,175.813 572.92,175.813 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 Soda/Adult 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip1704)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 32.2221,375.813 32.2221,375.806 46.2266,375.806 46.2266,375.697 60.231,375.697 60.231,374.876 74.2355,374.876 74.2355,370.64 88.24,370.64 88.24,354.212 
 102.244,354.212 102.244,323.94 116.249,323.94 116.249,257.441 130.253,257.441 130.253,231.496 144.258,231.496 144.258,239.263 158.262,239.263 158.262,285.576 
 172.267,285.576 172.267,331.341 186.271,331.341 186.271,358.081 200.276,358.081 200.276,370.902 214.28,370.902 214.28,374.687 228.285,374.687 228.285,375.579 
 242.289,375.579 242.289,375.781 256.294,375.781 256.294,375.807 270.298,375.807 270.298,375.813 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.5 
 
 
 4.0 
 
 
 4.5 
 
 
 5.0 
 
 
 5.5 
 
 
 6.0 
 
 
 6.5 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
 1.25 
 
 
 Soda/Kid 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip1705)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 335.517,375.813 335.517,375.805 349.594,375.805 349.594,375.743 363.671,375.743 363.671,375.346 377.748,375.346 377.748,372.674 391.825,372.674 391.825,362.974 
 405.902,362.974 405.902,337.092 419.979,337.092 419.979,294.802 434.056,294.802 434.056,264.722 448.132,264.722 448.132,250.556 462.209,250.556 462.209,293.678 
 476.286,293.678 476.286,336.383 490.363,336.383 490.363,362.567 504.44,362.567 504.44,372.667 518.517,372.667 518.517,375.176 532.594,375.176 532.594,375.747 
 546.671,375.747 546.671,375.808 560.748,375.808 560.748,375.813 574.825,375.813 574.825,375.813 
 "/>

Indeed we see that soda consumption is very high.

## Back Matter

In [71]:
versioninfo()

Julia Version 0.6.2
Commit d386e40c17 (2017-12-13 18:08 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin14.5.0)
  CPU: Intel(R) Core(TM) i7-4980HQ CPU @ 2.80GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Haswell)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, haswell)


When rendered correctly, this [Jupyter Notebook](http://jupyter.org/) uses the [Fira Code font](https://github.com/tonsky/FiraCode); a monospaced font with programming [ligatures](https://en.wikipedia.org/wiki/Typographic_ligature).

In [72]:
# Set up the notebook style
function css_styling()
    styles = open("custom.css") do f
        readstring(f)
    end
end
display(HTML(css_styling()))

HTML{String}("<style type=\"text/css\">\n    .container { width:100% !important; }\n    @font-face {\n       font-family: 'Fira Code';\n               src: url(\"https://cdn.rawgit.com/dunovank/jupyter-themes/1e851888/jupyterthemes/fonts/monospace/firacode/firacode.otf\") format(\"opentype\");\n    }\n   .CodeMirror {\n      font-family: 'Fira Code';\n      font-variant-ligatures: initial;\n    }\n\n   .cm-string {\n       font-variant-ligatures: none;\n    }\n \n    .input_area pre, .highlight pre {\n      font-family: 'Fira Code';\n      font-variant-ligatures: initial;\n   }\n\n   .highlight .err { border: 0px solid #FF0000 } /* Remove @> highlighting */\n\n\n</style>\n<script>\n    MathJax.Hub.Config({\n            TeX: {\n                extensions: [\"AMSmath.js\"]\n                           },\n                tex2jax: {\n                    inlineMath: [ ['\$','\$'], [\"\\\\(\",\"\\\\)\"] ],\n                    displayMath: [ ['\$\$','\$\$'], [\"\\\\[\",\"\\\\]\"] ]\n                },\n                displayAlign: 'center', // Change this to 'center' to center equations.\n                \"HTML-CSS\": {\n                    styles: {'.MathJax_Display': {\"margin\": 4}}\n                }\n        });\n</script>\n")